<h1 style="color:rgb(0,120,170)">344.038, KV Multimedia Search and Retrieval (WS2023/24)</h1>
<h2 style="color:rgb(0,120,170)">Task 3_Group B</h2>

| First Name | Family Name  | Matr.Nr   |
|:-----------|:-------------|:----------|
| Harald     | Eibensteiner | K01300179 |
| Hadi       | Sanaei       | K11733444 |
| Lukas      | Troyer       | K12006666 |
| Lukas      | Wagner       | K01357626 |
| Branko     | Paunović     | K12046370 |

# Retrieval systems

#### Import datasets, setup helper interfaces

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import numpy as np
np.random.seed(42)

from song import songs
from datasets import datasets
from retrieval import Retrieval
retrieval = Retrieval(n=100)
retrieval.precompute_all(threads=4)

In [3]:
# precompute retrievals (for frontend and chached lookup)

# only do this if you want to recompute the retrieval

retrieval = Retrieval(n=100)
retrieval.precompute_all(threads=4)

#### Prompt for user input

In [3]:
# Number of similar songs to retrieve
num_top_similar = int(input("Enter the number of top similar tracks: "))

# Example:
#   - Title: Letterman
#   - Artist: Wiz Khalifa
query, query_row_id = songs.prompt_song_query()

Enter the number of top similar tracks:  100
Enter the song title:  Letterman
Enter the artist name:  Wiz Khalifa


## Video-Based(\<similarity\>, \<feature\>)
### 1. Video-based(similarity, incp)

In [4]:
retN = Retrieval(n=num_top_similar)
Retrieval.create_df_from_tracks(retN.top_similar_tracks(query_row_id, dataset=datasets.resnet))

Error syncing cache with disk: Expecting ',' delimiter: line 1 column 37937813 (char 37937812)


,id,similarity,song,artist,album_name
0,GY25og83Ng5O9s75,0.783461,Remind Me,Röyksopp,Melody A.M.
1,gw9zZ3xtxoEjmYeB,0.777114,Locos,Cypress Hill,Elephants on Acid
2,8syTodUYnWUZtcIo,0.771750,Voodoo Child (Slight Return),Jimi Hendrix,Electric Ladyland
3,Dtf7ZV2mJSpyuvdD,0.770228,Hollow Man,R.E.M.,Accelerate
4,VSfGMu47DQFgMW4m,0.764065,Love Is The Drug,Grace Jones,Island Life
...,...,...,...,...,...
95,VtF32nX4mmVRGMEj,0.723284,Sweater Weather,The Neighbourhood,I Love You.
96,xE2N4kfcqEs2KAKR,0.722540,Quem Sabe Isso Quer Dizer Amor,Roberta Campos,Varrendo a Lua
97,syTw63wc7JVr6xLu,0.722504,Female Robbery,The Neighbourhood,I Love You.
98,xTkNXKaWbD5Koq8M,0.722439,Money,Leikeli47,Wash & Set


## Fusion-Based (\<similarity\>, \<feature\>)
### Early Fusion (musicnn & resnet)

In [2]:
import pandas as pd

def early_fusion_dataframe(df1, df2):
    if len(df1) != len(df2):
        raise ValueError("Both dataframes must have the same number of rows")

    # Concatenate dataframes along columns (axis=1)
    fused_df = pd.concat([df1, df2], axis=1)

    return fused_df


df_music = pd.read_csv('datasets/id_musicnn_mmsr.tsv', sep='\t')
df_resnet = pd.read_csv('datasets/id_resnet_mmsr.tsv', sep='\t')


df_ef = early_fusion_dataframe(df_music, df_resnet)


df_ef.to_csv("datasets/id_early_fusion_mmsr.tsv", sep='\t', index=False)



In [5]:
retN = Retrieval(n=num_top_similar)
Retrieval.create_df_from_tracks(retN.top_similar_tracks(query_row_id, dataset=datasets.early_fusion))

NameError: name 'num_top_similar' is not defined

In [11]:
# Perform tf_idf retrieval
df_tfidf = Retrieval.create_df_from_tracks(Retrieval(n=1000).top_similar_tracks(query_row_id,datasets.tf_idf))

# Perform musicnn retrieval
df_musicnn = Retrieval.create_df_from_tracks(Retrieval(n=1000).top_similar_tracks(query_row_id,datasets.musicnn))

# Perform resnet retrieval
df_resnet = Retrieval.create_df_from_tracks(Retrieval(n=1000).top_similar_tracks(query_row_id,datasets.resnet))



Error syncing cache with disk: Expecting ',' delimiter: line 1 column 37937813 (char 37937812)
Error syncing cache with disk: Expecting ',' delimiter: line 1 column 37937813 (char 37937812)
Error syncing cache with disk: Expecting ',' delimiter: line 1 column 37937813 (char 37937812)


# Late fusion - Score Aggregation Fusion for two DataFrames (text and audio)

In [12]:

# Late fusion - Score Aggregation Fusion for two DataFrames (text and audio)
def late_fusion_df(df_tfidf, df_musicnn, tfidf_weight=0.5, bert_weight=0.5):
    
    # Rename the 'similarity' column
    df_tfidf = df_tfidf.rename(columns={'similarity': 'similarity_tfidf'})
    df_musicnn = df_musicnn.rename(columns={'similarity': 'similarity_musicnn'})

    # Merge DataFrames on 'id'
    merged_df = pd.merge(df_musicnn[['id', 'similarity_musicnn']],df_tfidf, on='id')

    # Assuming the scores are already normalized between 0 and 1
    normalized_tfidf_scores = merged_df['similarity_tfidf'].values
    normalized_musicnn_scores = merged_df['similarity_musicnn'].values

    # Aggregate scores
    aggregated_scores = tfidf_weight * normalized_tfidf_scores + musicnn_weight * normalized_musicnn_scores

    # Add the aggregated scores to the DataFrame
    merged_df['aggregated_score'] = aggregated_scores


    # Sort DataFrame based on aggregated scores
    sorted_fusion_df = merged_df.sort_values(by='aggregated_score', ascending=False)

    return sorted_fusion_df

# Example usage
tfidf_weight = 0.6
musicnn_weight = 0.4
fusion_results_df = late_fusion_df(df_tfidf, df_musicnn, tfidf_weight, musicnn_weight)
fusion_results_df


,id,similarity_musicnn,similarity_tfidf,song,artist,album_name,aggregated_score
4,ezLpawJYbBm1YUwj,0.965511,0.249174,Slow,Liam Payne,First Time - EP,0.535709
2,iHRt8u88DH1Dq7SP,0.972653,0.236507,Loyal,Chris Brown,X (Expanded Edition),0.530966
0,Zcl514gBRFkd5xgR,0.984930,0.209372,Champion,Buju Banton,'Til Shiloh,0.519595
1,BvavHfrEqUNNphJD,0.978782,0.195008,Believe,Eminem,Revival,0.508518
3,tZjLHYdgOOsBESiG,0.966721,0.193732,Halftime,Nas,Illmatic,0.502928


# Late fusion - Rank-Level Fusion for two DataFrames (video and audio)


In [13]:

# Late fusion - Rank-Level Fusion for two DataFrames (video and audio)
def rank_level_fusion_video_audio(video_scores, audio_scores, video_weight=0.5, audio_weight=0.5):

    # Rename the 'similarity' column
    video_scores = video_scores.rename(columns={'similarity': 'similarity_video'})
    audio_scores = audio_scores.rename(columns={'similarity': 'similarity_audio'})

    # Merge DataFrames on 'id'
    merged_df = pd.merge(audio_scores[['id', 'similarity_audio']],video_scores, on='id')


     # Assuming the scores are already normalized between 0 and 1
    normalized_video_scores = merged_df['similarity_video'].values
    normalized_audio_scores = merged_df['similarity_audio'].values

    # Convert similarity scores to ranks using pandas Series
    video_ranks = pd.Series(normalized_video_scores).rank(ascending=False)
    audio_ranks = pd.Series(normalized_audio_scores).rank(ascending=False)

    # Rank-level fusion: Combine the ranks using weighted sum
    aggregated_ranks = video_weight * video_ranks + audio_weight * audio_ranks

    # Add the aggregated ranks to the DataFrame
    merged_df['aggregated_rank'] = aggregated_ranks

    # Sort DataFrame based on aggregated ranks
    sorted_fusion_df = merged_df.sort_values(by='aggregated_rank')

    return sorted_fusion_df

# Example usage
video_weight = 0.6
audio_weight = 0.4
fusion_results_df = rank_level_fusion_video_audio(df_resnet, df_musicnn, video_weight, audio_weight)

# Display the fused results
print("Rank-Level Fusion Results:")
print(fusion_results_df[['id', 'similarity_video', 'similarity_audio', 'aggregated_rank']])


Rank-Level Fusion Results:
                 id  similarity_video  similarity_audio  aggregated_rank
0  2IZjV6srFH5YXkFr          0.727367          0.987323              1.6
2  gw9zZ3xtxoEjmYeB          0.777114          0.954321              1.8
1  eVaVqLmvyuWX85Ll          0.725396          0.980981              2.6


# Evaluation 

In [ ]:
# Obtain necessary information and store for easy interop
from genres import Genres

genres = Genres()

## Accuracy
#### 1. Precision@k & Recall@k

In [ ]:
from precision_recall import PrecisionRecall

pr = PrecisionRecall(genres)
pr.compute()

In [ ]:
pr.plot_each()

In [ ]:
pr.plot_all_single()

#### 2. nDCG@10

In [ ]:
from ndcg import Ndcg

ndcg = Ndcg(genres)
ndcg.compute()
ndcg.plot()

## Beyond Accuracy
### 1. Genre Coverage@10

In [ ]:
from genre_coverage import GenreCoverage

genres_coverage = GenreCoverage(genres)
genres_coverage.compute()
genres_coverage.plot()

### 2. Genre Diversity@10

In [ ]:
from genre_diversity import GenreDiversity

genres_diversity = GenreDiversity(genres)
genres_diversity.compute()
genres_diversity.plot()

# Frontend

In [ ]:
from utils import write_song_df_to_json_file

write_song_df_to_json_file("frontend/static/songMeta.json", datasets.information.df, datasets.url.df, datasets.genres.df)

# frontend uses frontend/static/songMeta.json and retrievals/*.json
